In [16]:
import requests
from bs4 import BeautifulSoup
import json
import pandas
import time
import chardet
import random
import pandas as pd
import datetime

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
index_no = 4 # 0
place_infos = []
place_reviews = []

# 페이지 스크롤 함수
def scroll_page(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")  # 현재 페이지 높이
    while True:
        # 페이지 맨 아래로 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # 페이지 로딩 대기
        # 새로운 높이를 가져와서 이전 높이와 비교
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:  # 더 이상 스크롤할 내용이 없으면 종료
            break
        last_height = new_height

# 데이터 추출 함수
def extract_highlight_details(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    # 제목 추출
    title_element = soup.find('h2', class_='ModuleTitle-jljzy3-2 gLUazR')
    title = title_element.get_text(strip=True) if title_element else "No Title"
    # 설명 추출
    description_element = soup.find('div', class_='asctivity_details')
    description = description_element.get_text(strip=True) if description_element else "No Description"
    return title, description

def random_sleep(min_seconds=2, max_seconds=6):
    sleep_time = random.uniform(min_seconds, max_seconds)
    time.sleep(sleep_time)

list_url = 'https://us.trip.com/restapi/soa2/19913/getTripAttractionList'
review_url = 'https://www.trip.com/restapi/soa2/19707/getReviewSearch'

detail_headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
           'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
           'accept-encoding':'gzip, deflate, br, zstd',
           'accept-language':'en-US,en;q=0.9,ko;q=0.8'}

while True:
    # <1> 관광지 목록 - 상세페이지 url 찾기
    index_no += 1
    print('index_no: ', index_no) #####

    list_payload = {"head":{"extension":[{"name":"platform","value":"Online"},
                                         {"name":"locale","value":"en-US"},
                                         {"name":"currency","value":"USD"},
                                         {"name":"user-agent","value":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"}],
                    "cid":"1730772714280.178eQmti7ZqV"},
                    "scene":"gsDestination",
                    "districtId":234,
                    "index":index_no,
                    "count":10,
                    "sortType":1,
                    "returnModuleType":"product",
                    "filter":{"filterItems":[],"coordinateFilter":{"coordinateType":"wgs84","latitude":0,"longitude":0},"itemType":""},
                    "token":"Nzg5MTYsODMwMDEsNzg5MTAsMTA1MjQyNTUsMTQwNzc3MDIyLDMwNjMzNTE5LDg3NTU1LDEwNTIxNzU1LDUwNTgzNTA3LDEwNTQ4MjU5LDEwNzU4NjYxLDEzNjYyOTc1NSw4MTg1Myw1MDU3MTY0Niw4ODUxNyw4NzU2Myw4MTg1MiwxMTA3NzA5MSw5MDczNiw5OTgxMQ==",
                    "keyword":"",
                    "cityId":0,
                    "pageId":"10650006153",
                    "traceId":"a0128a91-132f-de79-4d65-1cb8d7426219"}
    try:
        list_resp = requests.post(list_url, json=list_payload)
        list_resp.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f'An error occured (list_resp): {e}')
        break

    list_encd = chardet.detect(list_resp.content)['encoding']

    list_js = json.loads(list_resp.content.decode(list_encd))
    place_list = list_js['attractionList']

    for place in place_list:
        if 'card' not in place or not place['card']['detailUrl']:
            continue

        place_info = {}

        # <2> 관광지 상세 - 관광지명, 평균 평점, 주소
        place_info['place_id'] = place['card']['poiId']
        place_info['place_name'] = place['card']['poiName']
        place_info['comm_score'] = place['card']['commentInfo']['commentScore']
        place_info['cover_image_url'] = place['card']['coverImageUrl']
        place_info['location'] = place['card']['location']
        print('place_id: ', place_info['place_id']) #####

        detail_url = place['card']['detailUrl']
        try:
            detail_resp = requests.get(detail_url, headers=detail_headers)
            detail_resp.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f'An error occured (detail_resp): {e}')
            break

        detail_soup = BeautifulSoup(detail_resp.content)
        place_info['address'] = detail_soup.select('.address-des-info .field')[0].text

        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
        driver.get(detail_url)
        try:
            # 페이지 스크롤
            scroll_page(driver)
            # 제목과 설명 데이터 추출
            title, description = extract_highlight_details(driver)
            # 결과 처리
            place_info['description'] = description
        finally:
            driver.quit()

        place_infos.append(place_info)

        # <3> trip.com review
        page_index = 0
        review_cnt = 0

        while True:
            page_index += 1

            review_payload = {"poiId":place_info['place_id'],
                              "locale":"en-US",
                              "pageSize":5,
                              "pageIndex":page_index,
                              "commentTagId":0,
                              "head":{"locale":"en-US",
                                      "cver":"3.0",
                                      "cid":"1730772714280.178eQmti7ZqV",
                                      "sid":"",
                                      "extension":[{"name":"locale","value":"en-US"},
                                                   {"name":"platform","value":"Online"},
                                                   {"name":"currency","value":"USD"},
                                                   {"name":"aid","value":""},
                                                   {"name":"user-agent","value":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"}]}}
            
            try:
                review_resp = requests.post(review_url, json=review_payload)
                review_resp.raise_for_status()
            except requests.exceptions.RequestException as e:
                print(f'An error occured (review_resp): {e}')
                break

            review_encd = chardet.detect(review_resp.content)['encoding']
            review_js = json.loads(review_resp.content.decode(review_encd))

            review_list = review_js['reviewList']
            review_count = review_js['reviewscount']

            for review in review_list:
                place_review = {}
                place_review['place_id'] = place_info['place_id']
                place_review['head_image'] = review['headImage'] if 'headImage' in review else ''
                place_review['user_name'] = review['username'] if 'username' in review else ''
                
                if 'en' in review['locale']:
                    place_review['content'] = review['content'] if 'content' in review else ''
                else:
                    place_review['content'] = review['translateContent'] if 'translateContent' in review else ''
                
                place_review['user_rating'] = review['userRating'] if 'userRating' in review else ''
                
                timestamp = review['createTime'] if 'createTime' in review else 0

                if timestamp != 0:
                    dt_obj = datetime.datetime.fromtimestamp(timestamp/1000)
                    place_review['create_time'] = dt_obj.strftime('%Y-%m-%d %H:%M:%S')
                else:
                    place_review['create_time'] = ''
                
                place_reviews.append(place_review)
                review_cnt += 1
                print(f'review_cnt: {review_cnt}/{review_count}')

                if review_cnt == 100:
                    break

            if review_cnt == 100 or review_cnt >= review_count:
                break

    df_pl_list = pd.DataFrame(place_infos)
    df_rev_list = pd.DataFrame(place_reviews)

    df_pl_list.to_csv(f'data/place_info_{index_no}.csv', index=False)
    df_rev_list.to_csv(f'data/place_review_{index_no}.csv', index=False)

    place_infos.clear()
    place_reviews.clear()

    if index_no == 300: # 300 페이지에서 Stop
        break

index_no:  5
place_id:  18700188
review_cnt: 1/29
review_cnt: 2/29
review_cnt: 3/29
review_cnt: 4/29
review_cnt: 5/29
review_cnt: 6/29
review_cnt: 7/29
review_cnt: 8/29
review_cnt: 9/29
review_cnt: 10/29
review_cnt: 11/29
review_cnt: 12/29
review_cnt: 13/29
review_cnt: 14/29
review_cnt: 15/29
review_cnt: 16/29
review_cnt: 17/29
review_cnt: 18/29
review_cnt: 19/29
review_cnt: 20/29
review_cnt: 21/29
review_cnt: 22/29
review_cnt: 23/29
review_cnt: 24/29
review_cnt: 25/29
review_cnt: 26/29
review_cnt: 27/29
review_cnt: 28/29
review_cnt: 29/29
review_cnt: 30/29
review_cnt: 31/29


KeyboardInterrupt: 

In [ ]:
# for detail
detail_headers = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
           'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
           'accept-encoding':'gzip, deflate, br, zstd',
           'accept-language':'en-US,en;q=0.9,ko;q=0.8'}
detail_url = 'https://www.trip.com/travel-guide/attraction/seoul/namsan-mountain-park-78916?curr=USD&locale=en-US'
detail_resp = requests.get(detail_url, headers=detail_headers)
detail_encd = chardet.detect(detail_resp.content)['encoding']

In [40]:
# for review
review_url = 'https://www.trip.com/restapi/soa2/19707/getReviewSearch'
review_payload = {"poiId":144940067,
                  "locale":"en-US",
                  "pageSize":5,
                  "pageIndex":1,
                  "commentTagId":0,
                  "head":{"locale":"en-US",
                          "cver":"3.0",
                          "cid":"1730772714280.178eQmti7ZqV",
                          "sid":"",
                          "extension":[{"name":"locale","value":"en-US"},
                                       {"name":"platform","value":"Online"},
                                       {"name":"currency","value":"USD"},
                                       {"name":"aid","value":""},
                                       {"name":"user-agent","value":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"}]}}
review_resp = requests.post(review_url, json=review_payload)

In [43]:
js = json.loads(review_resp.content)
js

{'ResponseStatus': {'Timestamp': '/Date(1730860786978+0800)/',
  'Ack': 'Success',
  'Extension': [{'Id': 'CLOGGING_TRACE_ID', 'Value': '986833820788728142'},
   {'Id': 'RootMessageId', 'Value': '100025527-0a93d031-480794-3458133'}]},
 'isSupportLocale': True,
 'score': 4.0,
 'reviewscount': 2,
 'pageIndex': 1,
 'reviewList': [{'reviewId': 194162109,
   'locale': 'zh-cn',
   'headImage': 'https://ak-d.tripcdn.com/images/1mi6o12000b5md1d77B69.png',
   'username': 'Anonymous User',
   'content': '值得去，但是但是不要在携程买票，买special seat，不是一个档次的体验',
   'userRating': 5.0,
   'createTime': 1727959813000,
   'helpfulLike': 1,
   'reviewImages': [],
   'thumbnailImageUrl': [],
   'source': '',
   'ischeck': 2,
   'totalReplyCount': 0,
   'replies': [],
   'hasVoted': False,
   'userUrl': '',
   'reviewUrl': '/rn_ibu_traveling/_crn_config?CRNModuleName=RN_Product_IBU_Traveling&CRNType=1&transparentstatusbar=1&initialPage=TripShootCommentDetail&commentId=194162109',
   'reviewH5Url': 'https://us.trip.com/

In [ ]:
# for tripadvisor review - not used
ta_comment_url = 'https://www.trip.com/restapi/soa2/19707/getTACommentList'
ta_comment_payload = {"arg":{"poiId":78916,
                             "resourceId":0,
                             "resourceType":0,
                             "sortType":0,
                             "sourceType":101,
                             "pageIndex":1,
                             "pageSize":10,
                             "locale":"en-US"},
                      "head":{"locale":"en-US",
                              "cver":"3.0",
                              "cid":"",
                              "sid":"",
                              "extension":[{"name":"locale","value":"en-US"},
                                           {"name":"platform","value":"Online"},
                                           {"name":"currency","value":"USD"},
                                           {"name":"aid","value":""},
                                           {"name":"user-agent","value":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"}]}}
ta_comment_resp = requests.post(ta_comment_url, json=ta_comment_payload)